# CH07-06: 三大模型架構對比
## Encoder vs Decoder vs Encoder-Decoder

**課程時長**: 75 分鐘  
**難度**: ⭐⭐⭐⭐  
**前置知識**: CH07-01 至 CH07-05  

---

## 📚 本節學習目標

1. ✅ 理解三大架構的設計哲學與適用場景
2. ✅ 掌握 BERT, GPT, T5 的核心差異
3. ✅ 學會根據任務選擇最佳架構
4. ✅ 理解預訓練任務對模型能力的影響
5. ✅ 實際對比三種架構的效能

---

## 📖 目錄

1. [三大架構概覽](#1-overview)
2. [Encoder-Only 架構 (BERT)](#2-encoder-only)
3. [Decoder-Only 架構 (GPT)](#3-decoder-only)
4. [Encoder-Decoder 架構 (T5)](#4-encoder-decoder)
5. [任務與架構的匹配](#5-task-matching)
6. [實戰對比與選型建議](#6-comparison)
7. [總結](#7-summary)

---

## 1. 三大架構概覽 {#1-overview}

### 1.1 架構分類

```
Transformer
    ├─ Encoder-Only (雙向理解)
    │   └─ BERT, RoBERTa, ALBERT, ELECTRA
    │
    ├─ Decoder-Only (單向生成)
    │   └─ GPT, GPT-2, GPT-3, LLaMA, Mistral
    │
    └─ Encoder-Decoder (序列轉換)
        └─ T5, BART, mBART, Pegasus
```

### 1.2 核心差異總覽

| 特性 | Encoder-Only | Decoder-Only | Encoder-Decoder |
|------|--------------|--------------|------------------|
| **注意力類型** | 雙向 Self-Attention | 單向 Masked Self-Attention | 雙向 (Encoder) + 單向 (Decoder) + Cross-Attention |
| **輸入-輸出** | 輸入 = 輸出長度 | 自回歸生成 | 輸入 ≠ 輸出長度 |
| **預訓練任務** | MLM, NSP | Causal LM | Span Corruption, Translation |
| **擅長任務** | 分類, NER, QA | 文本生成, 對話 | 翻譯, 摘要, 轉換 |
| **代表模型** | BERT | GPT | T5 |
| **參數效率** | 高 (理解) | 中 (生成) | 低 (需兩部分) |
| **推論速度** | 快 (並行) | 慢 (逐步生成) | 中 (Encoder並行, Decoder逐步) |

### 1.3 設計哲學

**Encoder-Only** (BERT 哲學):
> "理解語言的最佳方式是同時看到上下文的兩側"

**Decoder-Only** (GPT 哲學):
> "語言生成本質上是預測下一個詞，這個任務足以學會理解"

**Encoder-Decoder** (T5 哲學):
> "所有 NLP 任務都可以統一為 Text-to-Text 格式"

In [ ]:
# 載入必要套件
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from typing import List, Dict, Tuple
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 套件載入完成")

## 2. Encoder-Only 架構 (BERT) {#2-encoder-only}

### 2.1 架構特點

```
Input: "The cat [MASK] on the mat"
         ↓
    Token + Segment + Position Embeddings
         ↓
    Encoder Layer 1 (雙向 Self-Attention)
         ↓
    Encoder Layer 2
         ↓
         ...
         ↓
    Encoder Layer 12
         ↓
Output: Contextual Representations
         ↓
    Task-Specific Head (分類/NER/QA...)
```

### 2.2 BERT 系列模型

| 模型 | 層數 | d_model | 參數量 | 特點 |
|------|------|---------|--------|------|
| **BERT-Base** | 12 | 768 | 110M | 原始版本 |
| **BERT-Large** | 24 | 1024 | 340M | 更大容量 |
| **RoBERTa** | 12/24 | 768/1024 | 125M/355M | 移除 NSP, 更多數據 |
| **ALBERT** | 12/24 | 768/1024 | 12M/18M | 參數共享, 大幅減少參數 |
| **ELECTRA** | 12/24 | 768/1024 | 110M/335M | 改用 RTD 任務, 效率更高 |
| **DeBERTa** | 12/24 | 768/1024 | 140M/380M | Disentangled Attention |

### 2.3 預訓練任務

**① Masked Language Model (MLM)**:
```python
# 範例
Input:  "The cat [MASK] on the [MASK]"
Target: "The cat  sat   on the  mat "

# 遮蔽策略
- 80%: 替換為 [MASK]
- 10%: 替換為隨機詞
- 10%: 保持不變
```

**② Next Sentence Prediction (NSP)** (BERT 原始版本):
```python
# 正例
Sentence A: "I love NLP."
Sentence B: "It is very interesting."  # IsNext = True

# 負例
Sentence A: "I love NLP."
Sentence B: "The weather is nice today."  # IsNext = False
```

**RoBERTa 的改進**: 移除 NSP，只用 MLM，證明 NSP 對效果無益。

### 2.4 雙向注意力的優勢

**範例**: "The animal didn't cross the street because it was too tired."

- **單向 (Decoder)**: 處理 "it" 時只能看到左側，無法確定指代
- **雙向 (Encoder)**: 同時看到 "animal" 和 "tired"，能準確判斷 "it" = "animal"

In [ ]:
# 視覺化 BERT 的雙向注意力

def visualize_bidirectional_attention():
    """比較單向 vs 雙向注意力的可見範圍"""
    
    seq_len = 8
    tokens = ['The', 'cat', 'sat', 'on', 'the', 'mat', 'yesterday', '.']
    
    # 單向注意力 (Causal Mask)
    causal_mask = np.tril(np.ones((seq_len, seq_len)))
    
    # 雙向注意力 (No Mask)
    bidirectional_mask = np.ones((seq_len, seq_len))
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 單向注意力
    sns.heatmap(
        causal_mask, 
        annot=True, 
        fmt='.0f', 
        cmap='Blues',
        xticklabels=tokens,
        yticklabels=tokens,
        ax=axes[0],
        cbar_kws={'label': 'Can Attend'}
    )
    axes[0].set_title('Decoder-Only (GPT)\n單向注意力 - 只能看到過去', 
                      fontsize=13, fontweight='bold')
    axes[0].set_xlabel('Key Position')
    axes[0].set_ylabel('Query Position')
    
    # 雙向注意力
    sns.heatmap(
        bidirectional_mask, 
        annot=True, 
        fmt='.0f', 
        cmap='Greens',
        xticklabels=tokens,
        yticklabels=tokens,
        ax=axes[1],
        cbar_kws={'label': 'Can Attend'}
    )
    axes[1].set_title('Encoder-Only (BERT)\n雙向注意力 - 可以看到完整上下文', 
                      fontsize=13, fontweight='bold')
    axes[1].set_xlabel('Key Position')
    axes[1].set_ylabel('Query Position')
    
    plt.tight_layout()
    plt.show()
    
    print("\n關鍵差異:")
    print("• GPT: 處理 'sat' 時只能看到 [The, cat, sat]")
    print("• BERT: 處理 'sat' 時可以看到整個句子 [The ... .]")
    print("\n優勢:")
    print("• BERT 更適合理解任務 (分類、NER、問答)")
    print("• GPT 更適合生成任務 (文本生成、對話)")

visualize_bidirectional_attention()

### 2.5 BERT 的典型應用

**✅ 最適合的任務**:
1. **文本分類**: 情感分析、主題分類、垃圾郵件檢測
2. **命名實體識別 (NER)**: 人名、地名、組織名提取
3. **問答系統**: SQuAD, Natural Questions
4. **語義相似度**: 句子配對、文本蘊涵
5. **資訊檢索**: 搜尋排序、文檔相關性

**❌ 不適合的任務**:
- 文本生成 (需要 Decoder)
- 長文本摘要 (需要 Encoder-Decoder)
- 機器翻譯 (需要 Encoder-Decoder)

### 2.6 BERT 的優勢與限制

**優勢**:
- ✅ 雙向上下文理解最強
- ✅ 並行計算速度快
- ✅ 微調簡單高效
- ✅ 參數效率高

**限制**:
- ❌ 無法直接生成文本
- ❌ 預訓練與微調存在差距 ([MASK] 只出現在訓練時)
- ❌ 輸入長度受限 (通常 512 tokens)

## 3. Decoder-Only 架構 (GPT) {#3-decoder-only}

### 3.1 架構特點

```
Input: "The cat sat"
         ↓
    Token + Position Embeddings
         ↓
    Decoder Layer 1 (Masked Self-Attention)
         ↓
    Decoder Layer 2
         ↓
         ...
         ↓
    Decoder Layer N
         ↓
    Language Model Head
         ↓
Output: Next Token Probabilities → "on"
```

**關鍵**: 沒有 Cross-Attention，只有 Masked Self-Attention

### 3.2 GPT 系列演進

| 模型 | 層數 | d_model | 參數量 | 訓練數據 | 發布年份 |
|------|------|---------|--------|----------|----------|
| **GPT** | 12 | 768 | 117M | BooksCorpus (5GB) | 2018 |
| **GPT-2** | 48 | 1600 | 1.5B | WebText (40GB) | 2019 |
| **GPT-3** | 96 | 12288 | 175B | Common Crawl (570GB) | 2020 |
| **GPT-3.5** | - | - | ~175B | + Code + Instructions | 2022 |
| **GPT-4** | - | - | ~1.8T? | Multimodal | 2023 |

**開源替代品**:
- **LLaMA** (Meta): 7B/13B/33B/65B
- **Mistral** (Mistral AI): 7B, MoE 架構
- **Falcon** (TII): 7B/40B/180B

### 3.3 預訓練任務: Causal Language Modeling

**任務定義**: 給定前面的詞，預測下一個詞

```python
# 訓練範例
Input:  "The cat sat on"
Target: "the"

Input:  "The cat sat on the"
Target: "mat"

# 數學形式
P(x_t | x_1, x_2, ..., x_{t-1})

# 損失函數 (Negative Log-Likelihood)
Loss = -∑ log P(x_t | x_{<t})
```

### 3.4 自回歸生成範例

In [ ]:
def visualize_autoregressive_generation():
    """視覺化 GPT 的自回歸生成過程"""
    
    # 生成步驟
    steps = [
        {"input": "[BOS]", "context": ["[BOS]"], "predict": "I"},
        {"input": "I", "context": ["[BOS]", "I"], "predict": "love"},
        {"input": "love", "context": ["[BOS]", "I", "love"], "predict": "NLP"},
        {"input": "NLP", "context": ["[BOS]", "I", "love", "NLP"], "predict": "."},
        {"input": ".", "context": ["[BOS]", "I", "love", "NLP", "."], "predict": "[EOS]"},
    ]
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    y_positions = list(range(len(steps), 0, -1))
    
    for i, (step, y_pos) in enumerate(zip(steps, y_positions)):
        # 繪製上下文
        context_str = " ".join(step["context"])
        ax.text(0.1, y_pos, f"Step {i+1}", fontsize=12, fontweight='bold', va='center')
        ax.text(0.25, y_pos, f"Context: {context_str}", fontsize=11, va='center', 
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
        
        # 箭頭
        ax.annotate('', xy=(0.75, y_pos), xytext=(0.65, y_pos),
                   arrowprops=dict(arrowstyle='->', lw=2, color='green'))
        
        # 預測結果
        ax.text(0.8, y_pos, f'Predict: "{step["predict"]}"', fontsize=11, va='center',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, len(steps) + 1)
    ax.axis('off')
    ax.set_title('GPT 自回歸生成過程', fontsize=15, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()
    
    print("\n自回歸生成特點:")
    print("• 每步只預測一個 token")
    print("• 上下文逐步增長")
    print("• 每個 token 只能看到左側的歷史")
    print("• 生成長度不確定 (直到遇到 [EOS])")

visualize_autoregressive_generation()

### 3.5 GPT 的湧現能力 (Emergent Abilities)

隨著模型規模增長，GPT 展現出未經特別訓練的能力:

**GPT-3 的 Few-Shot Learning**:
```
# Zero-Shot (無範例)
Translate to French: "Hello" → 

# One-Shot (1 個範例)
Translate to French: "Hello" → "Bonjour"
Translate to French: "Goodbye" → 

# Few-Shot (多個範例)
Translate to French: "Hello" → "Bonjour"
Translate to French: "Thank you" → "Merci"
Translate to French: "Goodbye" → 
```

**In-Context Learning**: 模型在推論時從 prompt 中的範例學習，無需梯度更新。

### 3.6 GPT 的典型應用

**✅ 最適合的任務**:
1. **文本生成**: 故事、文章、詩歌創作
2. **對話系統**: ChatGPT, 客服機器人
3. **程式碼生成**: GitHub Copilot, CodeX
4. **文本補全**: IDE 自動補全、Email 續寫
5. **創意寫作**: 廣告文案、劇本

**⚠️ 需要 Prompt Engineering 的任務**:
- 分類 (需要設計 prompt 如 "Classify sentiment: positive or negative?")
- 資訊抽取 (需要示範格式)

### 3.7 GPT 的優勢與限制

**優勢**:
- ✅ 生成能力最強
- ✅ Zero/Few-shot 學習能力
- ✅ 統一架構適用多種任務
- ✅ 規模增長帶來能力提升

**限制**:
- ❌ 單向上下文理解較弱
- ❌ 推論慢 (逐步生成)
- ❌ 訓練成本極高
- ❌ 容易產生幻覺 (Hallucination)

## 4. Encoder-Decoder 架構 (T5) {#4-encoder-decoder}

### 4.1 架構特點

```
Source: "translate English to German: Hello"
           ↓
    Encoder (雙向理解)
           ↓
    Encoder Output ────┐
                       │
Target: "<BOS>"        │
           ↓           │
    Decoder (單向生成) │
           ↑           │
    Cross-Attention ←──┘
           ↓
Output: "Hallo"
```

**核心**: 結合 BERT 的理解能力 + GPT 的生成能力

### 4.2 T5 系列模型

| 模型 | Encoder 層數 | Decoder 層數 | d_model | 參數量 |
|------|--------------|--------------|---------|--------|
| **T5-Small** | 6 | 6 | 512 | 60M |
| **T5-Base** | 12 | 12 | 768 | 220M |
| **T5-Large** | 24 | 24 | 1024 | 770M |
| **T5-3B** | 24 | 24 | 1024 | 3B |
| **T5-11B** | 24 | 24 | 1024 | 11B |

**其他 Encoder-Decoder 模型**:
- **BART** (Facebook): Encoder-Decoder with denoising objectives
- **mBART** (Multilingual BART): 支援 50+ 語言
- **Pegasus** (Google): 專為摘要任務優化

### 4.3 T5 的統一框架: Text-to-Text

**核心思想**: 所有 NLP 任務都轉換為 "輸入文本 → 輸出文本"

```python
# 機器翻譯
Input:  "translate English to German: Hello"
Output: "Hallo"

# 文本分類
Input:  "sentiment: This movie is amazing!"
Output: "positive"

# 摘要
Input:  "summarize: [長篇文章...]"
Output: "[摘要...]"

# 問答
Input:  "question: What is NLP? context: [文章...]"
Output: "Natural Language Processing"

# 命名實體識別
Input:  "ner: John lives in New York"
Output: "John: PERSON, New York: LOCATION"
```

### 4.4 預訓練任務: Span Corruption

In [ ]:
def visualize_span_corruption():
    """視覺化 T5 的 Span Corruption 預訓練任務"""
    
    original = "Thank you for inviting me to your party last week"
    tokens = original.split()
    
    # 模擬 span corruption
    # Mask spans of varying lengths
    input_text = "Thank you <X> inviting <Y> to your party <Z> week"
    target_text = "<X> for <Y> me <Z> last"
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # 原始文本
    ax.text(0.5, 0.85, "原始文本", ha='center', fontsize=14, fontweight='bold')
    ax.text(0.5, 0.75, original, ha='center', fontsize=12,
            bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))
    
    # 箭頭
    ax.annotate('', xy=(0.5, 0.65), xytext=(0.5, 0.70),
               arrowprops=dict(arrowstyle='->', lw=2))
    ax.text(0.52, 0.67, 'Span Corruption', fontsize=10, style='italic')
    
    # Encoder Input
    ax.text(0.25, 0.55, "Encoder Input", ha='center', fontsize=13, fontweight='bold')
    ax.text(0.25, 0.45, input_text, ha='center', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.6))
    
    # Decoder Target
    ax.text(0.75, 0.55, "Decoder Target", ha='center', fontsize=13, fontweight='bold')
    ax.text(0.75, 0.45, target_text, ha='center', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.6))
    
    # 說明
    explanation = [
        "• 隨機選擇多個 spans (連續 token 片段) 進行遮蔽",
        "• 使用特殊 token <X>, <Y>, <Z> 替換被遮蔽的 spans",
        "• Decoder 需要依序重建被遮蔽的 spans",
        "• 比 BERT 的 MLM 更適合序列生成任務"
    ]
    
    y_start = 0.30
    for i, line in enumerate(explanation):
        ax.text(0.1, y_start - i*0.05, line, fontsize=10, va='top')
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('T5 預訓練任務: Span Corruption', fontsize=15, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_span_corruption()

print("\nSpan Corruption vs MLM (BERT):")
print("• BERT MLM: 遮蔽單個 token, 預測每個位置的 token")
print("• T5 Span Corruption: 遮蔽連續片段, 生成式地重建")
print("\n優勢: 更符合下游生成任務 (摘要、翻譯) 的需求")

### 4.5 T5 的典型應用

**✅ 最適合的任務**:
1. **機器翻譯**: 多語言互譯
2. **文本摘要**: 長文摘要、新聞標題生成
3. **問答系統**: 生成式問答 (而非抽取式)
4. **資料增強**: Paraphrase, Back-translation
5. **結構化輸出**: JSON 生成、格式轉換

**範例 - 多任務統一**:
```python
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# 任務 1: 翻譯
input_text = "translate English to French: Hello, how are you?"
# 任務 2: 摘要
input_text = "summarize: [長篇文章]"
# 任務 3: 問答
input_text = "question: What is NLP? context: [段落]"
```

### 4.6 T5 的優勢與限制

**優勢**:
- ✅ 統一框架簡化多任務學習
- ✅ 結合雙向理解 + 單向生成
- ✅ 適合需要深度理解的生成任務
- ✅ 靈活的輸入輸出長度

**限制**:
- ❌ 參數量大 (需要 Encoder + Decoder)
- ❌ 推論速度較慢
- ❌ 對 prompt 格式敏感
- ❌ 訓練成本高

## 5. 任務與架構的匹配 {#5-task-matching}

### 5.1 任務分類決策樹

```
你的任務需要生成文本嗎?
├─ 否 (理解任務)
│   ├─ 分類、NER、問答 (抽取式) → Encoder-Only (BERT)
│   └─ 語義相似度、資訊檢索 → Encoder-Only (BERT)
│
└─ 是 (生成任務)
    ├─ 輸入和輸出是否有明確對應?
    │   ├─ 是 (翻譯、摘要、轉換) → Encoder-Decoder (T5/BART)
    │   └─ 否 (續寫、對話、創作) → Decoder-Only (GPT)
    │
    └─ 需要深度理解源文本嗎?
        ├─ 是 (摘要、問答生成) → Encoder-Decoder (T5)
        └─ 否 (自由創作) → Decoder-Only (GPT)
```

### 5.2 任務-架構匹配表

In [ ]:
# 創建任務-架構匹配表
task_data = {
    '任務類型': [
        '情感分類',
        '命名實體識別',
        '問答 (抽取式)',
        '語義相似度',
        '機器翻譯',
        '文本摘要',
        '問答 (生成式)',
        '文本生成',
        '對話系統',
        '程式碼生成'
    ],
    'Encoder-Only (BERT)': [
        '✅ 最佳', '✅ 最佳', '✅ 最佳', '✅ 最佳',
        '❌ 不適合', '❌ 不適合', '⚠️ 可行但不佳',
        '❌ 不適合', '❌ 不適合', '❌ 不適合'
    ],
    'Decoder-Only (GPT)': [
        '⚠️ 需 Prompt', '⚠️ 需 Prompt', '✅ 可用',
        '⚠️ 需 Prompt', '✅ 可用', '✅ 可用', '✅ 最佳',
        '✅ 最佳', '✅ 最佳', '✅ 最佳'
    ],
    'Encoder-Decoder (T5)': [
        '✅ 可用', '✅ 可用', '✅ 可用', '⚠️ 較複雜',
        '✅ 最佳', '✅ 最佳', '✅ 最佳',
        '✅ 可用', '✅ 可用', '⚠️ 較複雜'
    ]
}

df = pd.DataFrame(task_data)

# 設置顏色對應
def color_cells(val):
    if '✅ 最佳' in val:
        return 'background-color: #90EE90; font-weight: bold'
    elif '✅' in val:
        return 'background-color: #FFFACD'
    elif '⚠️' in val:
        return 'background-color: #FFE4B5'
    elif '❌' in val:
        return 'background-color: #FFB6C1'
    return ''

# 顯示表格
styled_df = df.style.applymap(color_cells, subset=df.columns[1:])
display(styled_df)

print("\n圖例:")
print("✅ 最佳 - 此架構專為該任務設計")
print("✅ 可用 - 表現良好，但可能需要調整")
print("⚠️ 需 Prompt/較複雜 - 可行但需要特殊處理")
print("❌ 不適合 - 架構限制導致無法有效完成")

## 6. 實戰對比與選型建議 {#6-comparison}

### 6.1 效能對比 (相同參數量級)

**情境**: 假設都是 ~300M 參數的模型

In [ ]:
# 模擬效能對比數據
comparison_data = {
    '指標': [
        '文本分類 (GLUE)',
        '命名實體識別 (CoNLL)',
        '問答 (SQuAD 2.0)',
        '機器翻譯 (BLEU)',
        '文本摘要 (ROUGE)',
        '文本生成 (人工評分)',
        '訓練時間 (相對)',
        '推論速度 (tokens/sec)',
        '記憶體使用 (GB)'
    ],
    'BERT-Large': [92, 94, 88, 0, 0, 30, 1.0, 850, 12],
    'GPT-2-Medium': [85, 78, 82, 68, 72, 95, 0.8, 320, 10],
    'T5-Large': [91, 90, 90, 92, 88, 85, 1.5, 280, 18]
}

df_comparison = pd.DataFrame(comparison_data)

# 視覺化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 理解任務對比
understanding_tasks = df_comparison.iloc[:3, :]
x = np.arange(len(understanding_tasks['指標']))
width = 0.25

axes[0, 0].bar(x - width, understanding_tasks['BERT-Large'], width, label='BERT-Large', color='skyblue')
axes[0, 0].bar(x, understanding_tasks['GPT-2-Medium'], width, label='GPT-2-Medium', color='lightcoral')
axes[0, 0].bar(x + width, understanding_tasks['T5-Large'], width, label='T5-Large', color='lightgreen')
axes[0, 0].set_ylabel('分數', fontsize=11)
axes[0, 0].set_title('理解任務效能對比', fontsize=13, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(understanding_tasks['指標'], rotation=15, ha='right')
axes[0, 0].legend()
axes[0, 0].set_ylim(0, 100)
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. 生成任務對比
generation_tasks = df_comparison.iloc[3:6, :]
x = np.arange(len(generation_tasks['指標']))

axes[0, 1].bar(x - width, generation_tasks['BERT-Large'], width, label='BERT-Large', color='skyblue')
axes[0, 1].bar(x, generation_tasks['GPT-2-Medium'], width, label='GPT-2-Medium', color='lightcoral')
axes[0, 1].bar(x + width, generation_tasks['T5-Large'], width, label='T5-Large', color='lightgreen')
axes[0, 1].set_ylabel('分數', fontsize=11)
axes[0, 1].set_title('生成任務效能對比', fontsize=13, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(generation_tasks['指標'], rotation=15, ha='right')
axes[0, 1].legend()
axes[0, 1].set_ylim(0, 100)
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. 推論速度對比
speed_data = df_comparison.iloc[7:8, 1:].values[0]
models = ['BERT-Large', 'GPT-2-Medium', 'T5-Large']
colors_speed = ['skyblue', 'lightcoral', 'lightgreen']

axes[1, 0].barh(models, speed_data, color=colors_speed)
axes[1, 0].set_xlabel('Tokens/sec', fontsize=11)
axes[1, 0].set_title('推論速度對比 (越高越好)', fontsize=13, fontweight='bold')
axes[1, 0].grid(axis='x', alpha=0.3)

for i, v in enumerate(speed_data):
    axes[1, 0].text(v + 20, i, str(v), va='center', fontweight='bold')

# 4. 記憶體使用對比
memory_data = df_comparison.iloc[8:9, 1:].values[0]

axes[1, 1].barh(models, memory_data, color=colors_speed)
axes[1, 1].set_xlabel('記憶體 (GB)', fontsize=11)
axes[1, 1].set_title('記憶體使用對比 (越低越好)', fontsize=13, fontweight='bold')
axes[1, 1].grid(axis='x', alpha=0.3)

for i, v in enumerate(memory_data):
    axes[1, 1].text(v + 0.5, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n關鍵觀察:")
print("• BERT: 理解任務最強，推論最快，但無法生成")
print("• GPT: 生成任務最強，但理解任務需要 prompt engineering")
print("• T5: 平衡型選手，適合多任務場景，但資源消耗最大")

### 6.2 選型決策指南

#### 場景 1: 企業級文本分類系統

**需求**:
- 客戶評論情感分析
- 需要高準確率和快速推論
- 資料已標註

**推薦**: ✅ **BERT / RoBERTa**

**理由**:
1. 分類任務是 BERT 的強項
2. 微調簡單高效
3. 推論速度快，適合生產環境

---

#### 場景 2: AI 寫作助手

**需求**:
- 文章續寫、創意寫作
- 需要自然流暢的生成
- 支援多種風格

**推薦**: ✅ **GPT-3 / LLaMA / Mistral**

**理由**:
1. 生成能力最強
2. Few-shot learning 適應不同風格
3. 長文本生成連貫性好

---

#### 場景 3: 多語言新聞摘要系統

**需求**:
- 將長新聞摘要為短標題
- 支援多種語言
- 需要保持原文重點

**推薦**: ✅ **T5 / mBART / Pegasus**

**理由**:
1. Encoder-Decoder 專為序列轉換設計
2. Encoder 深度理解源文本
3. Decoder 生成簡潔摘要

---

#### 場景 4: 智能客服系統

**需求**:
- 理解用戶意圖
- 生成合適回覆
- 需要多輪對話能力

**推薦**: ✅ **GPT (小規模) + BERT (意圖分類)**

**理由**:
1. BERT 快速分類用戶意圖
2. GPT 生成自然回覆
3. 混合架構平衡效能與成本

---

### 6.3 實用選型檢查表

In [ ]:
def architecture_selector():
    """
    互動式架構選擇工具
    """
    print("=" * 60)
    print("        Transformer 架構選擇助手")
    print("=" * 60)
    print()
    
    questions = [
        {
            'q': '你的任務需要生成文本嗎？',
            'options': ['是 - 需要生成', '否 - 只需理解'],
            'scores': {'BERT': [0, 2], 'GPT': [2, 0], 'T5': [1, 1]}
        },
        {
            'q': '輸入和輸出是否有明確對應關係？(如翻譯、摘要)',
            'options': ['是 - 有對應', '否 - 自由生成'],
            'scores': {'BERT': [1, 0], 'GPT': [1, 2], 'T5': [2, 1]}
        },
        {
            'q': '推論速度是否為關鍵考量？',
            'options': ['是 - 需要快速推論', '否 - 品質優先'],
            'scores': {'BERT': [2, 1], 'GPT': [1, 1], 'T5': [0, 2]}
        },
        {
            'q': '是否有大量標註數據可供微調？',
            'options': ['是 - 有標註數據', '否 - 數據稀缺'],
            'scores': {'BERT': [2, 0], 'GPT': [1, 2], 'T5': [2, 1]}
        },
        {
            'q': '部署資源限制如何？',
            'options': ['受限 - 需要小模型', '充足 - 可用大模型'],
            'scores': {'BERT': [2, 1], 'GPT': [1, 1], 'T5': [0, 2]}
        }
    ]
    
    # 模擬用戶選擇 (實際使用時可以改為 input())
    user_choices = [0, 0, 0, 0, 1]  # 範例選擇
    
    scores = {'BERT': 0, 'GPT': 0, 'T5': 0}
    
    for i, (question, choice) in enumerate(zip(questions, user_choices)):
        print(f"Q{i+1}. {question['q']}")
        print(f"   選擇: {question['options'][choice]}")
        print()
        
        for arch in scores:
            scores[arch] += question['scores'][arch][choice]
    
    # 排序結果
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    print("=" * 60)
    print("推薦結果:")
    print("=" * 60)
    
    for rank, (arch, score) in enumerate(sorted_scores, 1):
        stars = '⭐' * score
        print(f"{rank}. {arch:8s} - {stars} ({score} 分)")
    
    best = sorted_scores[0][0]
    
    print("\n" + "=" * 60)
    print(f"✅ 建議使用: {best}")
    print("=" * 60)
    
    recommendations = {
        'BERT': '適合分類、NER、問答等理解任務，推論快速，易於微調',
        'GPT': '適合文本生成、對話系統，支援 few-shot learning',
        'T5': '適合翻譯、摘要等序列轉換任務，統一多任務框架'
    }
    
    print(f"\n理由: {recommendations[best]}")

architecture_selector()

## 7. 總結 {#7-summary}

### 7.1 三大架構核心要點

| 架構 | 核心優勢 | 最佳應用 | 代表模型 |
|------|----------|----------|----------|
| **Encoder-Only** | 雙向理解、快速推論 | 分類、NER、問答 | BERT, RoBERTa |
| **Decoder-Only** | 強大生成、Few-shot | 文本生成、對話 | GPT-3, LLaMA |
| **Encoder-Decoder** | 平衡理解與生成 | 翻譯、摘要 | T5, BART |

### 7.2 選型三原則

1. **任務導向**: 先確定任務類型(理解 vs 生成 vs 轉換)
2. **資源考量**: 評估計算資源、推論延遲需求
3. **數據情況**: 考慮標註數據量、是否需要 few-shot

### 7.3 未來趨勢

**架構融合**:
- **Instruction-tuned Models**: GPT-3.5, ChatGPT, Claude
  - Decoder-Only 架構 + 指令微調
  - 同時具備理解與生成能力

- **Mixture of Experts (MoE)**: Mistral 8x7B
  - 稀疏激活降低推論成本
  - 保持大模型能力

- **Multimodal Transformers**: GPT-4, Gemini
  - 跨模態理解(文本、圖像、音訊)

**實用建議**:

✅ **2024 年的最佳實踐**:
1. **理解任務**: 優先使用 DeBERTa, RoBERTa (效率高)
2. **生成任務**: 使用開源 LLM (LLaMA 2, Mistral) + Fine-tuning
3. **轉換任務**: T5 或 BART (專業領域可微調)
4. **通用場景**: 考慮 API (GPT-4, Claude) vs 自建模型的成本

### 7.4 延伸閱讀

1. **論文**:
   - [BERT: Pre-training of Deep Bidirectional Transformers](https://arxiv.org/abs/1810.04805)
   - [Language Models are Unsupervised Multitask Learners (GPT-2)](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)
   - [Exploring the Limits of Transfer Learning with T5](https://arxiv.org/abs/1910.10683)

2. **實戰資源**:
   - [Hugging Face Model Hub](https://huggingface.co/models)
   - [Papers with Code - NLP Leaderboards](https://paperswithcode.com/area/natural-language-processing)

---

## 🎯 下一節預告

**CH07-07: 大型語言模型 (LLMs)**
- LLM 的規模定律 (Scaling Laws)
- Instruction Tuning 與 RLHF
- Prompt Engineering 技巧
- In-Context Learning 原理
- LLM 的能力邊界與局限

---

**課程完成時間**: `____年____月____日`  
**學習心得**: ___________________________________